In [1]:
import pandas as pd
import numpy as np
from grouprecommender import GroupRecommender

# Loading the Dataset

In [2]:
poorly_formatted_rows = [2120259, 2446317, 11141080, 11152098, 11152401, 11882086, 12902538, 12935043, 17589538]
df = pd.read_csv('lastfm_data/userid-timestamp-artid-artname-traid-traname.tsv', header=None, skiprows=poorly_formatted_rows, sep='\t')
df.columns = ['user_id', 'timestamp', 'artist_id', 'artist_name', 'track_id', 'track_name']
df.dropna(inplace=True)

# Group Recommendations

In [3]:
gr = GroupRecommender('utility_matrix.pickle', df)

## Obtaining Group Recommendations

In [33]:
user_ids = np.random.choice(df['user_id'].unique(), 3, replace=False)
max_recommendations = 10
playlist = gr.full_recommendation(user_ids, max_recommendations, df, method='mean')

In [34]:
print("Recommended Songs for the Group:", user_ids)
for i, track in enumerate(playlist):
    print(str(i) + ')' + track[0] + ' - ' + track[1])

Recommended Songs for the Group: ['user_000296' 'user_000243' 'user_000519']
0)Haste - No Conditions
1)Deja-Vu - Önce
2)Deja-Vu - Alışık Değilsin
3)Paola & Chiara - Amoremidai
4)Thom Hell - Falling Down
5)Dauntless - My Guidance
6)Kotiteollisuus - Piru Irti
7)Neil Anderson - When Is It Gonna Happen
8)Emir Kusturica & The No Smoking Orchestra - Emir'S Dream
9)Dreamtale - Angels Eyes


## Evaluating Recommendations

In [11]:
#TODO: implement evaluation method
track_ids = [track.track_id for track in playlist]
playlist_indexes = np.where(np.in1d(df['track_id'].dropna().unique(), track_ids))[0].tolist()
user_indexes = np.where(np.in1d(df['user_id'].dropna().unique(), user_ids))[0].tolist()
gr.evaluate(user_indexes, playlist_indexes)


0.0 0


/Users/alex/Documents/itc/musicmagal/grouprecommender.py:135: RuntimeWarning: invalid value encountered in double_scalars
  


nan

In [32]:
# for track in playlist_indexes:
#     for user in user_indexes:
#         print(gr.utility_matrix[track, user])
sel = df[df['user_id'] == 'user_000072']
sel[sel['artist_name'] == 'Capercaillie']
#user_ids

,user_id,timestamp,artist_id,artist_name,track_id,track_name


## Creating Group Playlist

In [ ]:
import spotipy
import pprint
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '5de79a00f1b9475098f277ce2a609120'
client_secret = '909633d1415345fe89882b6e398390df'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                          client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
spotify_track_ids = []
for track in playlist:
    search_str = track[0] + ' ' + track[1]
    result = sp.search(search_str, limit=1)
    spotify_track_ids.append(result['tracks']['items'][0]['id'])

In [ ]:
playlist_name = 'Group recommendations for '
playlist_name += ', '.join(user_ids)

In [ ]:
username = '11130460071'
scope='playlist-modify-public'
token = util.prompt_for_user_token(username, 
                                   client_id=client_id, 
                                   client_secret=client_secret, 
                                   redirect_uri='https://github.com/alexing10/musicmagal',
                                   scope=scope)

In [ ]:
sp = spotipy.Spotify(auth=token)
playlist = sp.user_playlist_create(username, playlist_name)
sp.user_playlist_add_tracks(username, playlist['id'], spotify_track_ids)
print("boom!")

In [ ]:
sp.user_playlist_add_tracks(username, playlist_id, spotify_track_ids)